In [1]:
from common_functions import *
import pandas as pd 
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D,Conv2D,AveragePooling2D,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import LeakyReLU,PReLU
from keras.optimizers import SGD, RMSprop,Adam
import os
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
emotion = {'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3,
           'Sad': 4, 'Surprise': 5, 'Neutral': 6}
emo     = ['Angry', 'Fear', 'Happy',
           'Sad', 'Surprise', 'Neutral']
def reconstruct(pix_str, size=(48,48)):
    pix_arr = np.array(pix_str.split(' '),dtype=float)
    return pix_arr
def load_data(filepath='train.csv'):
    data = pd.read_csv(filepath)
    data['feature'] = data.feature.apply(lambda x: reconstruct(x))
    x = np.array([mat for mat in data.feature]).reshape(-1,48,48)# (n_samples, img_width, img_height)
    X_train = x.reshape(-1, 1, x.shape[1], x.shape[2])
    y_train=to_categorical(data['label'])
    print("X_train",X_train.shape,"y_train",y_train.shape)
    return (X_train, y_train)


In [3]:
X_train=np.load("raw_data/X_train.npy")
y_train=np.load("raw_data/y_train.npy")

In [4]:
#params:
batch_size = 100
epochs = 50
X_train=X_train[:28700]
y_train=y_train[:28700]
# setup info:
print ('X_train shape: ', X_train.shape) # (n_sample, 1, 48, 48)
print ('y_train shape: ', y_train.shape) # (n_sample, n_categories)
print( '  img size: ', X_train.shape[1], X_train.shape[2])
print ('batch size: ', batch_size)
print( '  nb_epoch: ', epochs)

X_train shape:  (28700, 48, 48, 1)
y_train shape:  (28700, 7)
  img size:  48 48
batch size:  100
  nb_epoch:  50


In [5]:
model = Sequential()
model.add(Conv2D(32,(3, 3), padding='same', activation='relu',
                        input_shape=( X_train.shape[1], X_train.shape[2],1)))
model.add(Conv2D(32,(3, 3), padding='same', activation='relu',))
model.add(Conv2D(32,(3, 3), padding='same', activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

# model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
# model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
# model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
# model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
# model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
# print 'Training....'
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
          validation_split=0.3, shuffle=True, verbose=1)

Train on 20090 samples, validate on 8610 samples
Epoch 1/50
20090/20090 [==============================] - 7s 353us/step - loss: 13.2883 - acc: 0.1724 - val_loss: 13.3256 - val_acc: 0.1731
Epoch 2/50
14500/20090 [====================>.........] - ETA: 0s - loss: 13.3709 - acc: 0.1703

KeyboardInterrupt: 

In [6]:
from keras.preprocessing.image import ImageDataGenerator

In [7]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [8]:
from sklearn.model_selection import train_test_split
# set vars


batch_size = 200
epochs = 150

X_train=np.load("raw_data/X_train.npy")
y_train=np.load("raw_data/y_train.npy")

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [9]:
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),validation_data=(X_val,y_val),
                    steps_per_epoch=len(X_train), epochs=epochs,verbose=1)

Epoch 1/150
  436/24402 [..............................] - ETA: 23:36 - loss: 1.7105 - acc: 0.3068

KeyboardInterrupt: 